In [1]:
import math
import numpy as np
import time as time
import random as rand
import cv2

In [2]:
class Wall:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        
    def draw(self, canvas):
        cv2.line(canvas, (int(self.x1), int(self.y1)), (int(self.x2), int(self.y2)), (255, 0, 0), 2)

In [3]:
def get_slope_interc(x1, y1, x2, y2):
    if abs(x2 - x1) < 0.0001:
        return None, x1 # store x offset in intercept (illegal but convenient)
    slope = (y2 - y1) / (x2 - x1)
    interc = y1 - slope * x1
    return slope, interc

def get_intersection(a1, b1, a2, b2):
    if a1 == a2 and b1 == b2:
        return 'everywhere', 'everywhere'
    elif a1 == a2:
        return None, None
    elif a1 == None:
        return b1, a2 * b1 + b2
    elif a2 == None:
        return b2, a1 * b2 + b1
    else:
        x_int = (b2 - b1) / (a1 - a2)
        y_int = a1 * x_int + b1
        return x_int, y_int
      
def in_between(val, low, high):
    return val >= low and val <= high or val >= high and val <= low
        
class Sensor:
    def __init__(self, rot=0): # rot relative to line: y = 0
        self.rot = rot % 360
        self.max_range = 50
        
    def rotate(self, rot):
        self.rot = (self.rot + rot) % 360
        
    def get_slope(self):
        if self.rot == 90:
            return None
        return math.tan(math.radians(self.rot))
        
        
    def get_closest_dist(self, x, y, walls):
        start_x, start_y, end_x, end_y = self.get_line(x, y)
        a1, b1 = get_slope_interc(start_x, start_y, end_x, end_y)
        min_wall = None
        min_dist = None
        for i, wall in enumerate(walls):
#             if i == 4:
#                 import pdb; pdb.set_trace()
            dist = None
            a2, b2 = get_slope_interc(wall.x1, wall.y1, wall.x2, wall.y2)
            x_int, y_int = get_intersection(a1, b1, a2, b2)
            if x_int == 'everywhere':
                # take one of end points of wall and calc distance to (x, y)
                dist1 = math.sqrt((wall.x1 - x)**2 + (wall.y1 - y)**2)
                dist2 = math.sqrt((wall.x2 - x)**2 + (wall.y2 - y)**2)
                dist = min(dist1, dist2)
            elif x_int == None:
                continue
            else:
                dist = math.sqrt((x_int - x)**2 + (y_int - y)**2)
            
            # check the intersection is in between wall, otherwise its not on wall and just on the line
            if dist != None and (min_dist is None or dist < min_dist) and in_between(x_int, wall.x1, wall.x2) and in_between(y_int, wall.y1, wall.y2) and (self.rot <= 180 and y_int <= y or self.rot > 180 and y_int >= y):
                min_wall = i
                min_dist = dist
                
#         print('Min wall: {}'.format(min_wall))
        return min_dist
        
    def get_line(self, x, y):
        rot_x = -self.max_range * math.cos(math.radians(self.rot))
        rot_y = -self.max_range * math.sin(math.radians(self.rot))
        end_x = x + rot_x
        end_y = y + rot_y
        return x, y, end_x, end_y
        
    def draw(self, x, y, canvas): #x, y are offsets for the line
        slope = self.get_slope()
        x1, y1, x2, y2 = self.get_line(x, y)
        cv2.line(canvas, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 1)

In [4]:
def get_mag(vec):
    sum = 0
    for val in vec:
        sum += val**2
        
    return math.sqrt(sum)

class Vehicle:
    def __init__(self, x, y, r=8, rot=0):
        self.x = x
        self.y = y
        self.radius = r
        self.alive = True
        self.mileage = 0
        
        # assume facing upwards initially
        self.sensors = [Sensor(rot=90), Sensor(rot=30), Sensor(rot=60), Sensor(rot=120), Sensor(150)]
        
        self.rotate(rot)
        
        # motion
        self.vel = [0, 0]
#         self.acc = [0, 0]
        
        self.limit = 1
        
        self.speed = self.limit
        
    def rotate(self, rot):
        for sensor in self.sensors:
            sensor.rotate(rot)
            
    def detect(self, walls):
        dists = []
        for sensor in self.sensors:
            dist = sensor.get_closest_dist(self.x, self.y, walls)
            if dist == None :
                dist = 10000 # arbitrary value to represent wall is super far away
            elif dist < self.radius:
                self.alive = False
            dists.append(dist)
        return dists
        
    def collision(self, walls):
        x = int(self.x)
        y = int(self.y)
        
        positions = []
        
        for r in range(y - self.radius, y + self.radius):
            for c in range(x - self.radius, x + self.radius):
                positions.append((c, r))
        
        for i, wall in enumerate(walls):
            x1, y1, x2, y2 = wall.x1, wall.y1, wall.x2, wall.y2
            a, b = get_slope_interc(x1, y1, x2, y2)
            for pos in positions:
                if abs(a * pos[0] + b - pos[1]) < 0.1 and in_between(pos[0], x1, x2) and in_between(pos[1], y1, y2):
                    return True
        return False
        
    def update(self):
        self.vel = [-self.speed * math.cos(math.radians(self.sensors[0].rot)), -self.speed * math.sin(math.radians(self.sensors[0].rot))]
        
        self.x = self.x + self.vel[0]
        self.y = self.y + self.vel[1]
        
        self.mileage += get_mag(self.vel)
        
        # reset
        self.acc = [0, 0]
        
    def draw(self, canvas):
        cv2.circle(canvas, (int(self.x), int(self.y)), self.radius, (255, 0, 0), -1)
        for sensor in self.sensors:
            sensor.draw(self.x, self.y, canvas)
        

In [5]:
class GeneticVehicle(Vehicle):
    def __init__(self, x, y, brain, rot=0):
        Vehicle.__init__(self, x, y, rot=rot)
        self.brain = brain
        
    def get_next_move(self, dists):
        # inputs => sensor distances (total of 5), speed
        inputs = dists
        
        # outputs => relative rot, acc mag
        rot = self.brain.feedforward(inputs)
        
        self.rotate(rot)

In [6]:
class Checkpoint:
    def __init__(self, x=None, y=None):
        self.x = x
        self.y = y
        
    @classmethod
    def create_checkpoint(cls, wall1, wall2):
        return Checkpoint((wall1.x1 + wall2.x1) // 2, (wall1.y1 + wall2.y1) // 2)
        

In [7]:
class Brain:
    def __init__(self, layer_sizes):
        self.input_size = layer_sizes[0]
        self.layer_sizes = layer_sizes[1:] # remove input size layer
        self.layers = []
        for i in range(0, len(layer_sizes) - 1):
            new_layer = np.array([[rand.uniform(-1, 1) for _ in range(layer_sizes[i] + 1)] for _ in range(layer_sizes[i + 1])])
            self.layers.append(new_layer)
        
    def mutate(self, mut_rate, mut_size):
        layer_sizes = [self.input_size] + self.layer_sizes
        new_brain = Brain(layer_sizes)
        for l in range(len(new_brain.layer_sizes)):
            for i in range(len(new_brain.layers[l])):
                for j in range(len(new_brain.layers[l][i])):
                    if rand.uniform(0, 1) < mut_rate:
                        new_brain.layers[l][i][j] += rand.uniform(-1, 1) * mut_size
        return new_brain
    
    def feedforward(self, inputs):
        cur_result = inputs + [1]
        for i in range(len(self.layers) - 1):
            next_result = np.array([math.tanh(np.dot(cur_result, self.layers[i][j])) for j in range(self.layers[i].shape[0])] + [1])
            cur_result = next_result
    
        output_result = np.array([math.tanh(np.dot(cur_result, self.layers[-1][j])) for j in range(self.layers[-1].shape[0])])
        
        return output_result

In [8]:
class GeneticPlayer():
    def __init__(self, pop_size=100, num_generations=100, num_iterations=200, start_pos=None, start_rot=0):
        self.pop_size = pop_size
        self.num_generations = num_generations
        self.num_iterations = num_iterations
        
        if start_pos == None:
            self.start_pos = (60, 500)
        else:
            self.start_pos = start_pos
            
        self.start_rot = start_rot
        
        input_size = 5
        hidden_size1 = 10
        hidden_size2 = 10
        output_size = 1
        self.layer_sizes = [input_size, hidden_size1, hidden_size2, output_size]

        # mutation
        self.mut_rate = 0.1
        self.mut_size = 0.1
        
        self.pop = [Brain(self.layer_sizes) for _ in range(self.pop_size)]
        
        self.last_best_brain = None
        self.last_best_score = None
            
    def run(self):
        max_score = 0
        scores = []
        # loop through each brain
        for i, brain in enumerate(self.pop):

            vehicle = GeneticVehicle(self.start_pos[0], self.start_pos[1], brain, rot=self.start_rot)
            game = Game(vehicle, num_iterations=self.num_iterations)
            game.run()
            # get score (= count)
            score = game.num_ckpts_passed + 10 / math.sqrt((vehicle.x - game.checkpoints[game.cur_ckpt_ind].x)**2 + (vehicle.y - game.checkpoints[game.cur_ckpt_ind].y)**2)
            scores.append(score)
            if score > max_score:
                self.last_best_brain = brain
                self.last_best_score = score
                max_score = score
#                 print('Highest score so far: {} for brain: {}'.format(score, i))
                print('Num ckpts passed: {}'.format(game.num_ckpts_passed))
            # store score
            
        # isolate for top scores anad select
        candid_indices = list(np.argsort(scores))[-self.pop_size // 4:] # get top 25%
        candids = [self.pop[i] for i in candid_indices][::-1] # reverse list so top brain is first
        candid_scores = [scores[i] for i in candid_indices][::-1]
        # mutate, pick randomly
        # set to current population
        self.pop = self.selection(candids, candid_scores)
        
    def selection(self, pop, scores):
        new_pop = []
        
        max_score = max(scores)
        mating_pool = []
        
        for i, brain in enumerate(pop):
            score = scores[i]
            num_elems = int(math.pow(score / max_score, 3) * 100)
            for _ in range(num_elems):
                mating_pool.append(brain)
                
        for i in range(self.pop_size // 3):
            r = rand.randint(0, len(mating_pool) - 1)
            chosen_brain = mating_pool[r]
            new_pop.append(chosen_brain.mutate(self.mut_rate, self.mut_size))
            new_pop.append(chosen_brain)
        
#         max_score = 0
        
#         for i in range(len(candids)):
#             new_pop.append(candids[i])
#             new_pop.append(candids[0].mutate(self.mut_rate, self.mut_size))
#             new_pop.append(candids[i].mutate(self.mut_rate, self.mut_size))
            
        for i in range(self.pop_size - len(new_pop)):
            new_pop.append(Brain(self.layer_sizes))
            
        return new_pop
        
    def evolve(self):
        # loop through each generation
        for i in range(self.num_generations):
            print('Generation: {}'.format(i))
            self.run()
            # run
    

In [27]:
DIST_THRESH = 100
class Game:
    def __init__(self, vehicle, num_iterations):
        self.vehicle = vehicle
        self.canvas_height, self.canvas_width = (600, 800)
        self.canvas = np.zeros((self.canvas_height, self.canvas_width))     
#         inner_walls = [Wall(83, 348, 256, 217), Wall(256, 217, 470, 211), Wall(470, 211, 575, 326), Wall(575, 326, 425, 470), Wall(425, 470, 133, 490), Wall(133, 490, 83, 348)]
#         outer_walls = [Wall(218, 158, 476, 149), Wall(476, 149, 684, 328), Wall(684, 328, 495, 551), Wall(495, 551, 40, 570), Wall(40, 570, 9, 301), Wall(9, 301, 218, 158)] 
#         wall_combs = [(5, 6), (0, 7), (1, 8), (2, 9), (3, 10), (4, 11)]
        outer_coors = ((67, 590), (38, 135), (343, 133), (347, 352), (211, 357), (217, 400), (382, 406), (378, 234), (485, 107), (748, 107), (582, 290), (590, 590))
        outer_walls = [Wall(outer_coors[i][0], outer_coors[i][1], outer_coors[(i + 1) % len(outer_coors)][0], outer_coors[(i + 1) % len(outer_coors)][1]) for i in range(len(outer_coors))]
        
        inner_coors = ((112, 523), (114, 205), (273, 200), (277, 299), (150, 303), (163, 480), (461, 473), (445, 238), (531, 157), (644, 157), (495, 266), (512, 510))
        inner_walls = [Wall(inner_coors[i][0], inner_coors[i][1], inner_coors[(i + 1) % len(inner_coors)][0], inner_coors[(i + 1) % len(inner_coors)][1]) for i in range(len(inner_coors))]
        
        wall_combs = [(0, 12), (1, 13), (2, 14), (3, 15), (4, 16), (5, 17), (6, 18), (7, 19), (8, 20), (9, 21), (10, 22), (11, 23)]
        self.walls = outer_walls + inner_walls    
    
        self.checkpoints = [Checkpoint.create_checkpoint(self.walls[inds[0]], self.walls[inds[1]]) for inds in wall_combs]
        
        self.num_iterations = num_iterations
        self.cur_ckpt_ind = 0
        self.num_ckpts_passed = 0
        
    def run(self, display=False):
        it = 0
        while True:
#             print(it)
            if self.num_iterations != None and it == self.num_iterations:
                break
            
            if display:
                self.canvas = np.zeros((self.canvas_height, self.canvas_width))     
                self.vehicle.draw(self.canvas)
                for i, ckpt in enumerate(self.checkpoints):
                    cv2.circle(self.canvas, (ckpt.x, ckpt.y), 10, (255, 0, 0), -1)
                    cv2.putText(self.canvas, str(i), (ckpt.x, ckpt.y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))
                for i, wall in enumerate(self.walls):
                    wall.draw(self.canvas)
                    cv2.putText(self.canvas, str(i), (wall.x1, wall.y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))
                cv2.imshow('canvas', self.canvas)
#                 cv2.imwrite('canvas.jpg', self.canvas)
                
            self.vehicle.update()
        
            if self.vehicle.collision(self.walls):
                self.vehicle.alive = False

            if self.vehicle.alive == False:
                break
#             print(math.sqrt((self.vehicle.x - self.checkpoints[self.cur_ckpt_ind].x)**2 + (self.vehicle.y - self.checkpoints[self.cur_ckpt_ind].y)**2))
            if math.sqrt((self.vehicle.x - self.checkpoints[self.cur_ckpt_ind].x)**2 + (self.vehicle.y - self.checkpoints[self.cur_ckpt_ind].y)**2) < DIST_THRESH:
#                 print(self.vehicle.x)
#                 print(self.vehicle.y)
#                 print(self.checkpoints[self.cur_ckpt_ind].x)
#                 print(self.checkpoints[self.cur_ckpt_ind].y)
#                 print(math.sqrt((self.vehicle.x - self.checkpoints[self.cur_ckpt_ind].x)**2 + (self.vehicle.y - self.checkpoints[self.cur_ckpt_ind].y)**2))
#                 print('Ckpt: {} passed'.format(self.cur_ckpt_ind))
                self.num_ckpts_passed += 1
                self.cur_ckpt_ind = (self.cur_ckpt_ind + 1) % len(self.checkpoints)
                
            dists = self.vehicle.detect(self.walls)
            self.vehicle.get_next_move(dists)

            if display:
                k = cv2.waitKey(3) & 0xFF
                if k == ord('q'):
                    break
                
            it += 1

        cv2.destroyAllWindows()

In [28]:
player = GeneticPlayer(pop_size=100, num_generations=10, start_pos=(340, 557), start_rot=-90)

# opt.
# player.pop[0] = biggest_brain

In [29]:
player.num_generations=10
player.num_iterations = None
player.evolve()

Generation: 0
Num ckpts passed: 1
Num ckpts passed: 1
Num ckpts passed: 1
Num ckpts passed: 6


KeyboardInterrupt: 

In [30]:
# start_pos = (434, 514)
start_pos = (340, 557)
biggest_brain = player.last_best_brain
vehicle = GeneticVehicle(start_pos[0], start_pos[1], brain=biggest_brain, rot=-90)
game = Game(vehicle, None)
game.run(display=True)

In [13]:
game.num_ckpts_passed

6